<a href="https://colab.research.google.com/github/pauhsainz/PRA3024/blob/main/GWA_Assignment_Paulina_Hernandez_Sainz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
!{sys.executable} -m pip install lalsuite pycbc


     |████████████████████████████████| 27.3MB 149kB/s 
     |████████████████████████████████| 3.2MB 28.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.6MB 34.9MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 481kB 39.9MB/s 
     |████████████████████████████████| 890kB 37.9MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 3.2MB 43.2MB/s 
  Created wheel for pycbc: filename=PyCBC-1.17.0-cp36-cp36m-linux_x86_64.whl size=5711452 sha256=abd1108cfa3e2d8f3f1003a8f51b30c6f17095184cf8d110cdd3c10fc48f60ba
  Stored in directory: /root/.cache/pip/wheels/51/1c/69/7de581c512e9465dad0d6a955c4a947425799fe14f6226d3a8
Successfully built pycbc
  Created wheel for lscsoft-glue: filename=lscsoft_glue-2.0.0-cp36-cp36m-linux_x86_64.whl size=414153 sha256=3fe893e32f8a245a3330a49fd98

In [ ]:
#Question 1 part 1
#Extracting data and functions from import files (Precious Preliminary Preparations :P)

%matplotlib inline
import pylab
import numpy
import numpy as np
import plotly
import plotly.graph_objects as go
import pandas
from pycbc.filter import highpass, matched_filter, resample_to_delta_t, sigma
from pycbc.catalog import Merger
from pycbc.frame import read_frame
from pycbc.psd import interpolate, inverse_spectrum_truncation
from pycbc.waveform import get_fd_waveform, get_td_waveform
from pycbc.conversions import mass1_from_mchirp_q


In [ ]:
merger = Merger("GW170817")
strain, stilde = {}, {}
for ifo in ['H1', 'L1']:
  ts = read_frame("{}-{}_LOSC_CLN_4_V1-1187007040-2048.gwf".format(ifo[0], ifo),
    '{}:LOSC-STRAIN'.format(ifo),
    start_time=merger.time - 224,
    end_time=merger.time + 32,
    check_integrity=False)

RuntimeError: ignored

In [ ]:
#printing the resulting time domain series
pylab.plot(strain['H1'].sample_times, strain['H1'])
pylab.xlabel('Time (s)')
pylab.show()

KeyError: ignored

In [ ]:
#Question 1 part 2 
#(Estimating PSD)

#Calculate the psd from the data -- Median Welch style estimate
#interpolate the PSD to the frequency step size we need.
psds = {}
for ifo in ['L1', 'H1']:
  psds[ifo] = interpolate(strain[ifo].psd(2), stilde[ifo].delta_f)
	
#Take care of corruption and overwhitening by manually setting a 2 second "buffer"
  psds[ifo] = inverse_spectrum_truncation(psds[ifo], int(2 * strain[ifo].sample_rate),
  low_frequency_cutoff=15.0,
  trunc_method='hann')

KeyError: ignored

In [ ]:
#Displaying our figures	
pylab.loglog(psds[ifo].sample_frequencies, psds[ifo], label=ifo)
pylab.xlim(20, 1024)
pylab.ylim(1e-47, 1e-42)
pylab.legend()

In [ ]:
#Setting mass range between 1.3 and 1.5 solar masses
masses = numpy.arange(1.3, 1.5, .01)

hmax, smax, tmax, mmax, nsnr = None, {}, {}, 0, 0
snrs = []
#Creating a template waveform using a Taylor approximation 
for m in masses:
	hp, hc = get_fd_waveform(approximant="TaylorF2", mass1=m, mass2=m, f_lower=20, 
                          delta_f=stilde[ifo].delta_f)
	hp.resize(len(stilde[ifo]))

KeyError: ignored

In [ ]:
#Looking for the point of maximum Signal to Noise ratio to look for possible matches with the template waveforms
max_snr, max_time = {}, {}
for ifo in ['L1', 'H1']:
		snr = matched_filter(hp, stilde[ifo], psd=psds[ifo],low_frequency_cutoff=20.0)

		snr = snr.time_slice(merger.time - 1, merger.time + 1)
		_, idx = snr.abs_max_loc()
		max_snr[ifo] = snr[idx]
#Find at what time this peak SNR occurs
		max_time[ifo] = float(idx) / snr.sample_rate + snr.start_time
network_snr = (abs(numpy.array(list(max_snr.values()))) ** 2.0).sum() ** 0.5 
snrs.append(max_snr)
#Selecting the maximum peak only
if network_snr > nsnr:
	tmax, hmax, mmax, smax = max_time, hp, m, max_snr
	nsnr = network_snr

# Finding where the peak SNR allows us (with the exception of redshift corrections at very large distances) to define what the masses that make up the system are 
print("We found that the best match for Mass1=Mass2 was %2.2f solar masses in the detector frame" % mmax)


NameError: ignored

In [ ]:
# print(snrs)
to_plot = []
for i in range(len(snrs)):
    to_plot.append((abs(snrs[i]['H1'])**2 + abs(snrs[i]['L1'])**2)**0.5)
# nsnr = (numpy.array(snrs['H1'])**2 + numpy.array(snrs['L1'])**2)**0.5
print(to_plot)

[]


In [ ]:
m = 1.4 # Solar masses
conditioned = strain['H1']
hp, hc = get_td_waveform(approximant="TaylorT2",
                     mass1=m,
                     mass2=m,
                     delta_t=conditioned.delta_t,
                     f_lower=20.0)

# We will resize the vector to match our data
hp.resize(len(conditioned))
#Shift the SNR time series towards the merger point and print a graph
template = hp .cyclic_time_shift(hp.start_time)
pylab.plot(hp.sample_times, template)
pylab.xlabel('time')
pylab.ylabel('strain')
pylab.show()

KeyError: ignored

In [ ]:
#Cropping data removes artifacts + extra 4 secs off the top
ifo = 'L1'                            
conditioned = strain[ifo]
snr = matched_filter(template, conditioned,
                     psd=psds[ifo] , low_frequency_cutoff=15)
snr = snr.crop(4 + 4, 4)

#plotting 
pylab.figure(figsize=[10, 4])
pylab.plot(snr.sample_times, abs(snr))
pylab.ylabel('Signal-to-noise')
pylab.xlabel('Time (s)')
pylab.show()

peak = abs(snr).numpy().argmax()
snrp = snr[peak]
time = snr.sample_times[peak]

print("We found a signal at {}s with SNR {}".format(time, 
                                                    abs(snrp)))

KeyError: ignored

In [ ]:
dt = time - conditioned.start_time
aligned = template.cyclic_time_shift(dt)
aligned /= sigma(aligned, psd=psds[ifo], low_frequency_cutoff=20.0)
aligned = (aligned.to_frequencyseries() * snrp).to_timeseries()
aligned.start_time = conditioned.start_time

white_data = (conditioned.to_frequencyseries() / psds[ifo]**0.5).to_timeseries()
tapered = aligned.highpass_fir(30, 512, remove_corrupted=False)
white_template = (tapered.to_frequencyseries() / psds[ifo]**0.5).to_timeseries()

white_data = white_data.highpass_fir(30., 512).lowpass_fir(300, 512)

white_data = white_data.time_slice(merger.time-.2, merger.time+.1)
white_template = white_template.time_slice(merger.time-.2, merger.time+.1)

pylab.figure(figsize=[15, 3])
pylab.plot(white_data.sample_times, white_data, label="Data")
pylab.plot(white_template.sample_times, white_template, label="Template")
pylab.legend()
pylab.show()


NameError: ignored

In [ ]:
subtracted = conditioned - aligned

# Plot the original data and the subtracted signal data

for data, title in [(conditioned, 'Original H1 Data'),
                    (subtracted, 'Signal Subtracted from H1 Data')]:

    t, f, p = data.whiten(4, 4).qtransform(.001,
                                                  logfsteps=100,
                                                  qrange=(8, 8),
                                                  frange=(20, 512))
    pylab.figure(figsize=[15, 3])
    pylab.title(title)
    pylab.pcolormesh(t, f, p**0.5, vmin=1, vmax=6)
    pylab.yscale('log')
    pylab.xlabel('Time (s)')
    pylab.ylabel('Frequency (Hz)')
    pylab.xlim(merger.time - 2, merger.time + 1)
    pylab.show()

NameError: ignored

In [ ]:
#Question 1 part 3

#Definitions
from pycbc.filter import match
from pycbc.psd import aLIGOZeroDetHighPower

#Defining two wavefunctions so they can be compared
hp, hc  =  get_td_waveform(approximant = "EOBNRv2",
                           mass1 = 10,
                           mass2 = 10,
                           f_lower = f_low,
                           delta_t = 1.0/sample_rate)
matches = []
masses = np.arange(5, 15, 0.2)

NameError: ignored

In [ ]:
for mass in masses:
    sp, sc = get_td_waveform(approximant = "TaylorT4",
                             mass1 = mass,
                             mass2 = mass,
                             f_lower = f_low,
                             delta_t = 1.0/sample_rate)

#Resizing the waveforms to the same length
    tlen = max(len(sp),len(hp))
    sp.resize(tlen)
    hp.resize(tlen)

NameError: ignored

In [ ]:
#Generate the aLIGO Zero det. high power PSD
delta_f = 1.0/sp.duration
flen = tlen // 2 + 1
psd = aLIGOZeroDetHighPower(flen, delta_f, f_low)
	
#Sifting through the wf's (SLOWWWWW)
m,i = match(hp, sp, psd=psd, low_frequency_cutoff = f_low)
matches.append(m)
	
#Plotting our GWWF against the templates
g1 = go.Figure()
fig1.add_trace(go.Scatter(x=masses,
                          y=matches,
                          mode='markers',
                          name="LIGO_Hanford"))
fig1.update_layout(xaxis_title='Mass',
                   yaxis_title='Match',
                   title="Correlation Between GW Waveform and Nearby Template")
fig2.write_html("Q1_8_waveform_correlation.html")

NameError: ignored

In [ ]:
IFrame(src='./Q1_8_waveform_correlation.html', width="100%", height=500)

In [ ]:
#Q2.1.1

#Imports
import numpy as np
import pycbc.types 

##Converting data into time series
d = np.load('noise_ts_4096Hz.npy')
dt = d[:, 0]
time = d[:,0]
strain = d[:,1]
d = pycbc.types.TimeSeries(d[:, 1], delta_t = dt[1]- dt[0])
data = d
strain = resample_to_delta_t(highpass(data, 20.0), 1.0/2048)
stilde = strain.to_frequencyseries()

FileNotFoundError: ignored

In [ ]:
##Estimate the PSD to frequency step 
psds = interpolate(strain.psd(2), stilde.delta_f)

## We'll choose how much data is corrupted by overwhitening
psds = inverse_spectrum_truncation(psds, 
                                   int(2 *strain.sample_rate),
                                   low_frequency_cutoff=15.0,
                                   trunc_method='hann')

AttributeError: ignored

In [ ]:
##Plotting our PSD								   
fig2 = go.Figure()
fig2.add_trace(go.Scattergl(x=psds.sample_frequencies,
                            y=psds,
                            mode='lines',
                            name="LIGO_Hanford"))
fig2.update_xaxes(type="log", range=[1.3,3])
fig2.update_yaxes(type="log", range=[-47,-42])
fig2.update_layout(xaxis_title='Frequency [Hz]',
                   yaxis_title='Amplitude',
                   title="Power Spectral Density")
fig2.write_html("Q2.1_Power_Spectral_Density.html")	

AttributeError: ignored

In [ ]:
IFrame(src='./Q2_1_Power_Spectral_Density.html', width="100%", height=500)

NameError: ignored

In [ ]:
#Q2.1.2
#MATCH FILTERING

#Defos
import pycbc.noise
import pycbc.psd
import pycbc.filter
import pycbc.waveform
import pylab
flow = 30 #Hz
stilde = data.to_frequencyseries()

#Generating template and rezising 
hp, hc = pycbc.waveform.get_fd_waveform(approximant="TaylorF2",
                             mass1=10, mass2=10,
                             f_lower=flow, delta_f=stilde.delta_f)

hp.resize(len(stilde))  
snr = pycbc.filter.matched_filter(hp, stilde, psd=psds,
                                      low_frequency_cutoff=flow)


# Removing the regions corrupted by filter wraparound
snr = snr[len(snr) // 4: len(snr) * 3 // 4]

NameError: ignored

In [ ]:
#Plotting our results, finding the max value of SNR 
pylab.plot(snr.sample_times, abs(snr))
pylab.ylabel('signal-to-noise ratio')
pylab.xlabel('time (s)')
pylab.show()
print ( 'Maximum SNR', max(abs(snr)) )

NameError: ignored

In [ ]:
IFrame(src='./Q2_2_SNR_Plot.html', width="100%", height=500)

NameError: ignored

In [ ]:
##Q2.2

#Defintitions
from pycbc.psd import welch, interpolate
import matplotlib.pyplot as plt

psds = interpolate(welch(data), 1.0 / data.duration) #???? fixed the issue of not same size
white_data = (data.to_frequencyseries() / psds**0.5).to_timeseries()

sigma = np.std(white_data)
mu = np.mean(white_data)

print("sigma = " + str(sigma),"; mu = " + str(mu))

NameError: ignored

In [ ]:
fig = go.Figure(data=[go.Histogram(x=white_data, histnorm='probability')])
fig.update_xaxes(range=[-200,200])
fig.update_layout(xaxis_title='noise',
                   yaxis_title='probability',
                   title="Whitened data histogram: &#963; = 60.4794879 and &#956; = 0.05229178")

fig.write_html("Q2_3_whiteNoise_Hist.html")

NameError: ignored

In [ ]:
IFrame(src='./Q2_3_whiteNoise_Hist.html', width="100%", height=500)

In [ ]:
##Q2.3

#Defos and generting wf

from pycbc.filter import sigmasq
hp, hc = get_fd_waveform(approximant="TaylorF2",
                         mass1=3,
                         mass2=3,
                         delta_f=0.001,
                         distance = 500,
                         f_lower=20.0, 
                         f_final = 2048.0) #Luminosity distance = 500 Mpc

nb_slice = 1000
slice_size = int(len(data)/nb_slice)

SNRs =[]

for i in range(nb_slice):
    data_slice = data[slice_size*i:slice_size*(i+1)]
#Calculating PSD    
    psds = interpolate(welch(data_slice), hp.delta_f)
    #print(psds.delta_f)
    SNR = (pycbc.filter.sigmasq(hp, psds))**0.5
    SNRs.append(SNR)
#Printing sigma & mu values
sigma = np.std(SNRs)
mu = np.mean(SNRs)
print("sigma = " + str(sigma),"; mu = " + str(mu))

NameError: ignored

In [ ]:
#plotting
fig = go.Figure(data=[go.Histogram(x=SNRs, histnorm='probability')])
# fig.update_xaxes(range=[-0,200])
fig.update_layout(xaxis_title='SNR',
                   yaxis_title='probability',
                   title="Estimated SNR histogram: &#963; = 1.6560736 and &#956; = 15.6536511")
fig.write_html("Q2_4_estSNR_Hist.html")

NameError: ignored

In [ ]:
IFrame(src='./Q2_4_estSNR_Hist.html', width="100%", height=500)

NameError: ignored

##Q3
To be honest this assignment, even with the extra time, was overwhelming. The instructions for this question elude me in the technical aspect, mainly because I don't understand what I'm expected to calculate. However, the theory aspect of telescope threshold sensitivity I do understand more.
The horizon distance is the "area of sight" of the telescope where it can detect SNRs above 8. This horizon decreases after a 500 solar mass threshold, because more massive binaries would produce gravitational waves at frequencies so high that the sensitivity of the detector would not be sufficient to register and isolate them.In order to measure such high frequencies accurately it becomes most important to measure accurately smaller and smaller discrete pieces of space, over a larger distance. This is why new generation telescopes are important for further research, by becoming more precise, more noise isolated and larger, they will be able to unlock a world of higher frequency Gravitational events out there, as well as help measure the currently detectble ones from a different geographical position and precision. 

Of course then this brings the problem of analysis, this whole thing we are trying to do in the first place. Our methods and tools for analysis will also have to improve massively to accomodate for this incoming bountiful harvest of gravitational waves.
No easy answers on how to solve this yet, but we can certainly try.
